### 얼굴인식(Face Recognition)

- face_recognition(dlib) ==> 사진 속 인물이 DB 인물 중 누군지 자동 매칭
- DeepFace ==> 얼굴 인식 + 성별, 감정, 나이, 인종 추정까지
- FaceNet ==> 얼굴을 임베딩 벡터로 변환해서 비교 가능

### 어떤 인물인지, 얼마나 닮았는지 비교

#### 단체사진 얼굴 인식 및 임베딩 추출

In [12]:
from deepface import DeepFace
import cv2
import numpy as np 

try:
    faces = DeepFace.extract_faces(
        "target_group_photo.jpg",
        detector_backend='retinaface', 
        enforce_detection=False
    )
except Exception as e:
    print(f"얼굴 추출 중 오류 발생: {e}")
    faces = [] 

face_embeddings = []

for i, face_obj in enumerate(faces):
    if "face" not in face_obj:
        print(f"{i+1}번째 객체에 'face' 데이터가 없습니다.")
        continue

    face_img = face_obj["face"]  

    if face_img.size == 0:
        print(f"{i+1}번째 얼굴 이미지가 비어있습니다.")
        continue

    try:
        if face_img.dtype == "float64" or face_img.dtype == "float32":
            face_img = (face_img * 255).astype("uint8")

        embedding_obj = DeepFace.represent(
            img_path=face_img,
            model_name="Facenet",
            enforce_detection=False 
        )
        embedding = embedding_obj[0]["embedding"]
        face_embeddings.append(embedding)

        bgr_face = cv2.cvtColor(face_img, cv2.COLOR_RGB2BGR)
        cv2.imwrite(f"face_{i+1}.jpg", bgr_face)
        print(f"face_{i+1}.jpg 저장 및 임베딩 완료.")

    except Exception as e:
        print(f"{i+1}번째 얼굴 처리 실패:", e)

face_1.jpg 저장 및 임베딩 완료.
face_2.jpg 저장 및 임베딩 완료.
face_3.jpg 저장 및 임베딩 완료.
face_4.jpg 저장 및 임베딩 완료.
face_5.jpg 저장 및 임베딩 완료.


#### 개인 사진 비교 후 "누구인지" 추정

In [ ]:
from scipy.spatial import distance
import cv2 
import numpy as np 
import os 

TARGET_PHOTO_PATH = "target_photo5.png" 


# 1. 매칭할 대상 사진에서 얼굴 특징(임베딩) 추출
try:
    print(f"'{TARGET_PHOTO_PATH}'에서 얼굴을 분석합니다...")
    target_embedding_obj = DeepFace.represent(
        img_path=TARGET_PHOTO_PATH,
        model_name="Facenet",
        enforce_detection=True
    )
    target_embedding = target_embedding_obj[0]["embedding"]
    print("대상 사진의 얼굴 분석 완료.")

    # 2. 저장된 모든 얼굴 임베딩과 코사인 거리 계산
    distances = []
    for emb in face_embeddings:
        dist = distance.cosine(target_embedding, emb)
        distances.append(dist)

    # 3. 계산된 거리 중 가장 짧은 값(가장 닮은 얼굴) 찾기
    if not distances:
        print("분석할 얼굴 데이터가 없습니다. 첫 번째 셀을 먼저 실행하세요.")
    else:
        min_distance_index = np.argmin(distances)
        min_distance = distances[min_distance_index]
        matched_face_number = min_distance_index + 1

        print("\n--- 🔍 최종 분석 결과 ---")
        print(f"사진 속 인물은 'face_{matched_face_number}.jpg'와 가장 닮았습니다.")
        print(f"(유사도 점수: {min_distance:.4f})")

        if min_distance < 0.4:
            print("신뢰도: 매우 높음 (동일인일 확률이 높습니다)")
        else:
            print("신뢰도: 보통 (닮은 사람이지만, 다른 인물일 수 있습니다)")

        #  4. 결과 이미지 생성 및 저장 (추가된 부분) 
        try:
            # 원본 타겟 이미지와 매칭된 얼굴 이미지를 불러옴
            target_img = cv2.imread(TARGET_PHOTO_PATH)
            matched_face_filename = f"face_{matched_face_number}.jpg"
            matched_img = cv2.imread(matched_face_filename)

            # 두 이미지를 나란히 붙이려면 높이를 맞춰야 함
            # 타겟 이미지의 높이를 기준으로 매칭된 얼굴 이미지의 크기를 조절
            h1, w1, _ = target_img.shape
            h2, w2, _ = matched_img.shape
            
            # 비율을 유지하면서 매칭된 이미지의 높이를 타겟 이미지 높이에 맞춤
            scale_ratio = h1 / h2
            new_w2 = int(w2 * scale_ratio)
            resized_matched_img = cv2.resize(matched_img, (new_w2, h1))

            # 두 이미지를 가로로 합치기
            combined_img = cv2.hconcat([target_img, resized_matched_img])
            
            # 결과 이미지에 텍스트로 유사도 점수 추가
            score_text = f"Score: {min_distance:.4f}"
            org = (10, 30) # 텍스트 위치 (좌측 상단)
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(combined_img, score_text, org, font, 1, (0, 255, 0), 2, cv2.LINE_AA)

            # 최종 결과 이미지를 파일로 저장
            result_filename = f"result_match.jpg"
            cv2.imwrite(result_filename, combined_img)
            print(f"비교 결과 이미지를 '{result_filename}'로 저장했습니다.")

        except Exception as e:
            print(f"결과 이미지 생성 중 오류 발생: {e}")


except ValueError:
    print(f"오류: '{TARGET_PHOTO_PATH}'에서 얼굴을 찾지 못했습니다. 다른 사진으로 시도해보세요.")
except Exception as e:
    print(f"오류가 발생했습니다: {e}")

'target_photo5.png'에서 얼굴을 분석합니다...
✅ 대상 사진의 얼굴 분석 완료.

--- 🔍 최종 분석 결과 ---
사진 속 인물은 'face_4.jpg'와 가장 닮았습니다.
(유사도 점수: 0.5887)
신뢰도: 🟡 보통 (닮은 사람이지만, 다른 인물일 수 있습니다)
✅ 비교 결과 이미지를 'result_match.jpg'로 저장했습니다.


### 여러명 사진 비교 후 저장

In [ ]:
from scipy.spatial import distance
from scipy.optimize import linear_sum_assignment # 헝가리안 알고리즘을 위한 임포트
import cv2
import numpy as np
import os
from deepface import DeepFace

# --- 설정 ---
TARGET_IMAGE_FILES = [
    "target_photo5.jpg",
    "target_photo6.png",
    "target_photo7.png",
    "target_photo8.png",
    "target_photo9.png",
]
STANDARD_SIZE = (150, 150)
DISTANCE_THRESHOLD = 0.5 # 이 값보다 거리가 멀면 매칭에서 제외 

# --- 1. 모든 타겟 이미지의 임베딩 미리 계산하기 ---
target_embeddings = []
valid_target_paths = [] # 얼굴 추출에 성공한 파일 경로만 저장
print("모든 타겟 이미지 분석 시작")
for target_path in TARGET_IMAGE_FILES:
    if not os.path.exists(target_path):
        print(f"파일 없음: {target_path}")
        continue
    try:
        embedding_obj = DeepFace.represent(img_path=target_path, model_name="Facenet", enforce_detection=True)
        target_embeddings.append(embedding_obj[0]["embedding"])
        valid_target_paths.append(target_path) # 성공한 경우에만 리스트에 추가
        print(f"분석 성공: {target_path}")
    except ValueError:
        print(f"얼굴 찾기 실패: {target_path}")
    except Exception as e:
        print(f"오류 발생: {target_path} ({e})")

# --- 2. 비용 행렬(Cost Matrix) 생성 ---
# 행: 타겟 얼굴, 열: DB 얼굴, 값: 두 얼굴 간의 코사인 거리
num_targets = len(target_embeddings)
num_db_faces = len(face_embeddings)
cost_matrix = np.zeros((num_targets, num_db_faces))

for i in range(num_targets):
    for j in range(num_db_faces):
        cost_matrix[i, j] = distance.cosine(target_embeddings[i], face_embeddings[j])

# --- 3. 헝가리안 알고리즘으로 최적의 매칭 찾기 ---
# linear_sum_assignment는 비용을 최소화하는 최적의 (타겟, DB얼굴) 짝의 인덱스를 반환
target_indices, db_indices = linear_sum_assignment(cost_matrix)
print("\n--- 🔍 최적 매칭 결과 분석 ---")

# --- 4. 최종 결과 이미지 생성 ---
combined_results_list = []
for target_idx, db_idx in zip(target_indices, db_indices):
    matched_distance = cost_matrix[target_idx, db_idx]
    
    # 임계값(Threshold)을 통과한, 신뢰도 높은 매칭만 결과에 포함
    if matched_distance < DISTANCE_THRESHOLD:
        target_path = valid_target_paths[target_idx]
        matched_face_number = db_idx + 1
        
        print(f"'{target_path}' face_{matched_face_number}.jpg (유사도: {matched_distance:.4f})")
        
        # 시각적 결과물 생성
        target_img_bgr = cv2.imread(target_path)
        matched_img = cv2.imread(f"face_{matched_face_number}.jpg")
        
        resized_target = cv2.resize(target_img_bgr, STANDARD_SIZE)
        resized_matched = cv2.resize(matched_img, STANDARD_SIZE)
        
        file_basename = os.path.basename(target_path)
        score_text = f"{file_basename} ({matched_distance:.2f})"
        color = (0, 255, 0) # 신뢰도 높은 매칭은 녹색으로 표시
        cv2.putText(resized_target, score_text, (5, STANDARD_SIZE[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

        pair_image = cv2.hconcat([resized_target, resized_matched])
        combined_results_list.append(pair_image)
    else:
        target_path = valid_target_paths[target_idx]
        print(f"'{target_path}'의 매칭 결과는 신뢰도가 낮아 제외합니다. (유사도: {matched_distance:.4f})")


if combined_results_list:
    final_image = cv2.vconcat(combined_results_list)
    result_filename = "result_optimal_matches.jpg"
    cv2.imwrite(result_filename, final_image)
    print(f"\n중복 없는 최적 매칭 결과 이미지를 '{result_filename}'로 저장했습니다.")
else:
    print("\n신뢰도 높은 매칭 결과가 없어 최종 이미지를 생성하지 않았습니다.")

--- 👤 모든 타겟 이미지 분석 시작 ---
✅ 분석 성공: target_photo5.jpg
✅ 분석 성공: target_photo6.png
✅ 분석 성공: target_photo7.png
✅ 분석 성공: target_photo8.png
✅ 분석 성공: target_photo9.png

--- 🔍 최적 매칭 결과 분석 ---
'target_photo5.jpg' ↔️ face_1.jpg (유사도: 0.2322)
'target_photo6.png' ↔️ face_4.jpg (유사도: 0.4088)
'target_photo7.png' ↔️ face_3.jpg (유사도: 0.0931)
'target_photo8.png' ↔️ face_2.jpg (유사도: 0.3170)
'target_photo9.png' ↔️ face_5.jpg (유사도: 0.3417)

🎉 중복 없는 최적 매칭 결과 이미지를 'result_optimal_matches.jpg'로 저장했습니다.
